In [1]:
from __future__ import division
from hmmlearn.hmm import MultinomialHMM
import pandas as pd
import numpy as np

from  Preprocesscopy import Preprocesscopy

from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

In [2]:
file_path = '/home/rui/Documents/MLProject/data/modified-crab/trips-in-sanfran.csv'
city = 'san_francisco'
tile_size = 0.360
la_size = 0.003234
long_size = 0.004049
start_id = 0
freq_threshold = 0

In [3]:
data = Preprocesscopy(file_path, city, tile_size, la_size, long_size, freq_threshold)

In [4]:
data.read_dataset()
print "done read dataset"
data.split_train_test()
print "done split dataset"
data.find_significant_region(start_id)
print "done find significant region"
data.filter_low_frequency_trajectory()
print "done filter low frequency traj"
data.refind_significant_region(start_id)
print "done refind significant region"
data.convert_train_traj_to_region_id()
print "done convert train traj to region id"
data.convert_test_traj_to_region_id()
print "done convert test traj to region id"

done read dataset
done split dataset
done find significant region
done filter low frequency traj
done refind significant region
done convert train traj to region id
done convert test traj to region id


In [5]:
taxi_train = data.taxi_train
taxi_test = data.taxi_test

In [ ]:
taxi_test

In [10]:
hmm  = joblib.load("hiddenmarkovmodel360-20.pkl")  

In [11]:
transmat = hmm.transmat_
emissionprob = hmm.emissionprob_
startprob = np.column_stack([hmm.startprob_])
H = 20
predictions = []

correct_prediction = 0
total_prediction = 0


for row_index, row in taxi_test.iterrows():
    x = row[3]
    T = len(x)
    
    # alpha recursion
    alpha = np.zeros((H, T))
    alpha = np.asmatrix(alpha)
    z = np.zeros(T)  # local normalisaton factors
    phgh = transmat.T
    
    alpha[:, 0] = np.row_stack(emissionprob[:, x[0]]) * startprob
    z[0] = alpha[:, 0].sum()
    alpha[:, 0] = alpha[:, 0] / z[0]
    next_hidden = phgh.dot(alpha[:, 0])
    next_observation = emissionprob.T.dot(next_hidden)
    next_prediction = np.where(next_observation == np.max(next_observation))
    total_prediction += 1
    if next_prediction[0][0] == x[1]:
            correct_prediction += 1
    
    
    for t in range(1, T-1):
        predictor = phgh.dot(alpha[:, t-1])
        alpha[:, t] = np.row_stack(emissionprob[:, x[t]]) * np.array(predictor)
        z[t] = alpha[:, t].sum()
        alpha[:, t] = alpha[:, t] / z[t]
        next_hidden = phgh.dot(alpha[:, 0])
        next_observation = emissionprob.T.dot(next_hidden)
        next_prediction = np.where(next_observation == np.max(next_observation))
        total_prediction += 1
        predictions.append(next_prediction[0][0])
        if next_prediction[0][0] == x[t+1]:
            correct_prediction += 1

accuracy = correct_prediction / total_prediction
print "The accuracy is %f " %(accuracy)

The accuracy is 0.058187 


4 hidden states: 0.038463 

8 hidden states: 0.041145 

12 hidden states:  0.052145 

16 hidden states: 0.055166 

20 hidden states: 0.058187 

24 hidden states: 0.053594 